# Linear Regression

In [6]:
import mysql.connector
from mysql.connector import errorcode
import config
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 1000

%matplotlib inline

In [14]:
dbname = 'nba'
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    db = dbname
)
cursor = cnx.cursor(buffered = True)

## Preprocessing

### Helper functions

#### Function to convert minutes to seconds

In [15]:
def calc_secs(x):
    return int(x.split(':')[0])*60 + int(x.split(':')[1])

In [16]:
def sql_to_df(table, year=''):
    
    if year:
        year_clean = '00' + year[0] + year[2:]
        select_statement = "select * from nba.{} where (GAME_ID REGEXP '(^{})') order by GAME_ID".format(table, year_clean)
    else:
        select_statement = "SELECT * FROM nba.{} order by GAME_ID;".format(table)
    
    cursor.execute(select_statement)
    data = cursor.fetchall()
    data_df = pd.DataFrame(data)
    
    field_names = [i[0] for i in cursor.description]
    data_df.columns = field_names
    
    return data_df

### Aggregation functions

#### Function to calculate rolling statistics for players from box score data

In [19]:
def formated_player_stats(table, cols, year = '', lag = None):
    
    nba_df = sql_to_df(table, year)
    
    ### Drop games where players do not play
    nba_df_played = nba_df[nba_df['MIN'].notnull()]
    
    ### Calculate seconds played in a game
    nba_df_played['SEC'] = nba_df_played['MIN'].apply(lambda x : calc_secs(x))
    
    
    players = []
    cols = cols
    for player in nba_df_played['PLAYER_ID'].unique():
        df = nba_df_played[nba_df_played['PLAYER_ID']==player]
        for col in cols:
            df['expanding_{}'.format(col)] = df.expanding().mean()[col].shift(1)
            if lag:
                df['rolling_{}'.format(col+str(lag))] = df.rolling(lag).mean()[col].shift(1)
        df.dropna(inplace=True)
        players.append(df)
    return pd.concat(players)

#### Function to calculate rolling team statistics

In [ ]:
def formatted_team_stats(table, cols, year = '', lag = None):
    
    pull = sql_to_df(table, year)
    
    teams_df = []
    cols = cols
    ### For every team, calculate the expanding mean for the given columns passed into the function, 
    ### and the the difference between the statistic in the game and the expanding mean leading up to that game
    ### Answers the question "how did a team perform compared to its average going into the game"
    for team in pull['TEAM_ID'].unique():
        df = pull[pull['TEAM_ID']==team]
        df['game_number'] = df['GAME_ID'].expanding().count()
        for col in cols:
            df['expanding_{}'.format(col)] = df.expanding().mean()[col].shift(1)
            df['difference_{}'.format(col)] = df[col] - df['expanding_{}'.format(col)]
            ### Add ability to see how team performs compared to their rolling average over a set of games
            ### Answers question "how did a team perform compared to its average over previous x number of games"
            if lag:
                df['rolling_{}'.format(col+str(lag))] = df.rolling(lag).mean()[col].shift(1)
                df['lag_difference_{}'.format(col)] = df[col] - df['rolling_{}'.format(col+str(lag))]
            continue
        df.dropna(inplace=True)
        teams_df.append(df)
        
        
    return pd.concat(teams_df)